In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd #pandas library
from pandas_datareader import data #data readers (google, html, etc.)

import numpy as np
import datetime as dt #datetime for timeseries support


api_key = 'put_api_here'

In [2]:

df = pd.read_csv("destinations_LP_crawler_Ex5.csv",encoding='latin1')
# print(df['city'])

countries=[]


for city in df['city']:

    url_geocode="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (city,api_key)
   
    try:
        response_geocode = requests.get(url_geocode) 
        if not response_geocode.status_code == 200:
            print("HTTP error")
        else:
            try:
               #geocode
                response_geocode=response_geocode.json()
                country=response_geocode['results'][0]['formatted_address'].split(',')[-1].strip()
                try:
                    #sometimes number appear at the end of formatted_address, if so, take earler word
                    int(country)
                    country=response_geocode['results'][0]['formatted_address'].split(',')[-2].strip()
                except:
                    country=country
                    
                countries.append(country)
#                 print(country)


            except: 
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
# print (countries)
# print(len( df['city']),"country",len(countries))


In [3]:
df['country']=countries
df

,city,city_LP,country
0,Antalya,mediterranean-coast/antalya,Turkey
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain
...,...,...,...
71,Venice,Venice,Italy
72,Vienna,Vienna,Austria
73,Warsaw,Warsaw,Poland
74,Zakynthos,Zakynthos,Greece


In [4]:
Description=[]
bace='https://www.lonelyplanet.com/'
for i in range(len(df['city'])):
    url=bace+df['country'][i]+'/'+df['city_LP'][i]
    response = requests.get(url)
    

    if not response.status_code == 200:
        print("Error",url)
        Description.append('Error')
    else: 
        result = BeautifulSoup(response.content,'html.parser')
        description= result.find('div',class_='readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg')
        Description.append(description.get_text())
    


In [5]:
del df['city_LP']
df['description']=Description
df.to_csv('LP_destinations.csv')



In [6]:
df

,city,country,description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
71,Venice,Italy,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."
